**ML PROBLEM 4.1**

Alexandre Reis nº100123

Rodolfo Amorim nº100260

In [74]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.linear_model import LassoCV, Lasso
from sklearn import linear_model 
import matplotlib.pyplot as plt
import numpy as np
from math import *
import os


**Loading the data from the files given**

In [65]:
X_train = np.load(os.path.abspath('X_train_regression1.npy'))
y_train = np.load(os.path.abspath('y_train_regression1.npy'))
X_test  = np.load(os.path.abspath('X_test_regression1.npy'))

**Centering of the data**

In [66]:
X_train_mean  = np.mean(X_train,axis=0) 
y_train_mean  = np.mean(y_train,axis=0)

X_train_prime = X_train.copy()
y_train_prime = y_train.copy()

for i in range(10):
    X_train_prime[:,i] = X_train_prime[:,i] -  X_train_mean[i]

# y_train_prime is a one dimensional array, and as such we do not include it in the for loop 
y_train_prime = y_train_prime - y_train_mean 

**Application of the LASSO Regularization**


In [67]:
LassoModelCV = LassoCV(alphas=np.arange(0,10,0.01),cv=8,fit_intercept=False,random_state=0).fit(X_train_prime, y_train_prime) 
LassoModel = Lasso(LassoModelCV.alpha_).fit(X_train_prime, y_train_prime) 
beta_hat_lasso  = LassoModel.coef_ # Beta Coefficients from the LASSO Regularization


# Removal of the unnecessary columns from the X data
X_train_prime_lasso_ridge= X_train_prime.copy()
X_test_lasso_ridge = X_test.copy()
for i in range(np.shape(beta_hat_lasso)[0]-1, -1, -1):
    if beta_hat_lasso[i]==0:
        X_train_prime_lasso_ridge = np.delete(X_train_prime_lasso_ridge, i, 1)
        X_test_lasso_ridge = np.delete(X_test_lasso_ridge, i, 1)
        X_train_mean = np.delete(X_train_mean, i, axis = 0)

c:\Users\rodol\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\rodol\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:614: UserWarning: Coordinate descent without L1 regularization may lead to unexpected results and is discouraged. Set l1_ratio > 0 to add L1 regularization.
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\rodol\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.058196398721421, tolerance: 0.006776555683252925
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\rodol\AppData\Local\Progr

**Aplication of the Ridge Regularization to the LASSO Regularization Coefficients**

In [68]:
RidgeModel = linear_model.RidgeCV(alphas=np.arange(0.01,10,0.01), fit_intercept=False, scoring="neg_mean_squared_error", cv=5).fit(X_train_prime_lasso_ridge, y_train_prime)
beta_hat_ridge= RidgeModel.coef_  # Beta Coefficients from the application of the Ridge Regularization to the Coefficients from the LASSO Regularization

**Calculation of the Beta Coefficients to make predictions on Uncentered data**

In [69]:
beta_zero = y_train_mean - np.matmul((X_train_mean),np.transpose(beta_hat_ridge))
m = [beta_zero[0]]
for k in range(((beta_hat_ridge).shape[1])):
        m.append(beta_hat_ridge[:,k][0])
beta_hat_lasso_ridge = np.array(m)

**Prediction for the Test dataset**

In [73]:
X_test_final = np.hstack((np.ones((1000,1)),X_test_lasso_ridge))
prediction = np.matmul(X_test_final,beta_hat_lasso_ridge)
prediciton = prediction.reshape(1000,1)
np.save('prediction_100123_100260_4.1',prediction)